In [4]:

import collections
import copy
import hashlib
import io
import os
import subprocess
import textwrap
import time

from typing import List, Text

from PIL import Image

import numpy as np
import pandas as pd
import tabulate


import tensorflow as tf

import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'tabulate'

In [1]:
from google.cloud import storage

client = storage.Client()   # will use ADC automatically
print(client.project)

/labs/khatrilab/provido/miniconda_2026/envs/processing/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


OSError: Project was not passed and could not be determined from the environment.

In [2]:
class Globals:
  # GCP project with GCS bucket of interest
  gcp_project = 'dx-scin-public' #@param

  # GCS bucket with data to read
  gcs_bucket_name = 'dx-scin-public-data' #@param

  # CSV of case metadata to read
  cases_csv = 'dataset/scin_cases.csv' #@param

  # CSV of label metadata to read
  labels_csv = 'dataset/scin_labels.csv' #@param

  # Images directory
  gcs_images_dir = 'dataset/images/' #@param

  ### Key column names
  image_path_columns = ['image_1_path', 'image_2_path', 'image_3_path']
  weighted_skin_condition_label = "weighted_skin_condition_label"
  skin_condition_label = "dermatologist_skin_condition_on_label_name"

  ###### Formed during execution:

  # Client for querying GCS
  gcs_storage_client = None

  # Bucket object for loading files
  gcs_bucket = None

  # pd.DataFrame for the loaded metadata_csv
  cases_df = None

  # pd.DataFrame for the loaded labels_csv
  cases_and_labels_df = None

print(f'GCS bucket name: {Globals.gcs_bucket_name}')
print(f'cases_csv: {Globals.cases_csv}')
print(f'labels_csv: {Globals.labels_csv}')
print(f'images dir: {Globals.gcs_images_dir}')

GCS bucket name: dx-scin-public-data
cases_csv: dataset/scin_cases.csv
labels_csv: dataset/scin_labels.csv
images dir: dataset/images/


In [5]:
from google.cloud import storage

def list_blobs(storage_client, bucket_name):
  """Helper to list blobs in a bucket (useful for debugging)."""
  blobs = storage_client.list_blobs(bucket_name)
  for blob in blobs:
    print(blob)

def initialize_df_with_metadata(bucket, csv_path):
  """Loads the given CSV into a pd.DataFrame."""
  df = pd.read_csv(io.BytesIO(bucket.blob(csv_path).download_as_string()), dtype={'case_id': str})
  df['case_id'] = df['case_id'].astype(str)
  return df

def augment_metadata_with_labels(df, bucket, csv_path):
  """Loads the given CSV into a pd.DataFrame."""
  labels_df = pd.read_csv(io.BytesIO(bucket.blob(csv_path).download_as_string()), dtype={'case_id': str})
  labels_df['case_id'] = labels_df['case_id'].astype(str)
  merged_df = pd.merge(df, labels_df, on='case_id')
  return merged_df

Globals.gcs_storage_client = storage.Client(Globals.gcp_project)
Globals.gcs_bucket = Globals.gcs_storage_client.bucket(
    Globals.gcs_bucket_name
)
Globals.cases_df = initialize_df_with_metadata(Globals.gcs_bucket, Globals.cases_csv)
Globals.cases_and_labels_df = augment_metadata_with_labels(Globals.cases_df, Globals.gcs_bucket, Globals.labels_csv)
print(len(Globals.cases_and_labels_df))

/labs/khatrilab/provido/miniconda_2026/envs/processing/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


5033


In [11]:
import pandas as pd

df = pd.read_csv("/home/provido/scin_meta/scin_cases.csv")
df.columns
# Most useful single label
#df["weighted_skin_condition_label"].value_counts(dropna=False).head(30)

# Raw dermatologist label
#df["dermatologist_skin_condition_on_label_name"].value_counts(dropna=False).head(30)

Index(['case_id', 'source', 'release', 'year', 'age_group', 'sex_at_birth',
       'fitzpatrick_skin_type',
       'race_ethnicity_american_indian_or_alaska_native',
       'race_ethnicity_asian', 'race_ethnicity_black_or_african_american',
       'race_ethnicity_hispanic_latino_or_spanish_origin',
       'race_ethnicity_middle_eastern_or_north_african',
       'race_ethnicity_native_hawaiian_or_pacific_islander',
       'race_ethnicity_white', 'race_ethnicity_other_race',
       'race_ethnicity_prefer_not_to_answer', 'textures_raised_or_bumpy',
       'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled',
       'body_parts_head_or_neck', 'body_parts_arm', 'body_parts_palm',
       'body_parts_back_of_hand', 'body_parts_torso_front',
       'body_parts_torso_back', 'body_parts_genitalia_or_groin',
       'body_parts_buttocks', 'body_parts_leg', 'body_parts_foot_top_or_side',
       'body_parts_foot_sole', 'body_parts_other',
       'condition_symptoms_bothersome_appearan

In [21]:
df2 = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/dataset_scin_labels.csv")
df2['dermatologist_skin_condition_on_label_name'].tolist()

["['Inflicted skin lesions', 'Eczema', 'Irritant Contact Dermatitis']",
 "['Prurigo nodularis', 'Actinic Keratosis', 'SCC/SCCIS']",
 "['Impetigo', 'Herpes Zoster', 'Bullous dermatitis']",
 '[]',
 "['Lichen planus/lichenoid eruption', 'Folliculitis', 'Dermatosis due to flea']",
 "['Drug Rash', 'Viral Exanthem', 'Eczema']",
 "['Urticaria', 'Leukocytoclastic Vasculitis', 'Urticaria', 'Insect Bite', 'Leukocytoclastic Vasculitis']",
 "['Inflicted skin lesions', 'Basal Cell Carcinoma', 'SCC/SCCIS']",
 "['Psoriasis', 'Eczema', 'Eczema', 'Acute and chronic dermatitis', 'Eczema', 'Post-Inflammatory hyperpigmentation']",
 "['Purpura']",
 "['Drug Rash', 'Eczema', 'Allergic Contact Dermatitis']",
 "['Urticaria']",
 '[]',
 "['Skin infection', 'Scabies', 'Molluscum Contagiosum']",
 '[]',
 "['Urticaria', 'Tinea Versicolor', 'Urticaria', 'Pityriasis rubra pilaris', 'Tinea Versicolor', 'Psoriasis']",
 '[]',
 "['Stasis Dermatitis', 'Eczema', 'Allergic Contact Dermatitis']",
 "['Tinea Versicolor', 'Sebor

In [19]:
import pandas as pd
import ast
import re

def classify_diagnosis(diagnosis_list_str):
    """
    Classify patient diagnoses into target labels using regex pattern matching.
    
    Args:
        diagnosis_list_str: String representation of a list of diagnoses
        
    Returns:
        str: One of the target labels or 'unclear'
    """
    # Parse the string representation of list
    try:
        if diagnosis_list_str == '[]' or pd.isna(diagnosis_list_str):
            return 'unclear'
        diagnosis_list = ast.literal_eval(diagnosis_list_str)
    except:
        return 'unclear'
    
    if not diagnosis_list:
        return 'unclear'
    
    # Combine all diagnoses into a single string for pattern matching
    combined_diagnoses = ' '.join(diagnosis_list).lower()
    
    # Define regex patterns for each target label
    # Using word boundaries (\b) to ensure we match whole words
    patterns = {
        'melanoma': r'\bmelanoma\b',
        'dermatitis': r'\bdermatitis\b',
        'scleroderma': r'\bscleroderma\b',
        'psoriasis': r'\bpsoriasis\b',
        'systemic lupus erythematosus': r'\b(systemic\s+lupus\s+erythematosus|lupus\s+erythematosus|sle)\b',
        'vitiligo': r'\bvitiligo\b'
    }
    
    # Check each pattern
    for label, pattern in patterns.items():
        if re.search(pattern, combined_diagnoses):
            return label
    
    return 'unclear'

# Apply the classification
df2 = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/dataset_scin_labels.csv")
df2['classified_condition'] = df2['dermatologist_skin_condition_on_label_name'].apply(classify_diagnosis)

# Show results
print("Classification Results:")
print(df2['classified_condition'].value_counts())

print("\nSample classifications:")
for idx, row in df2.head(25).iterrows():
    print(f"{row['dermatologist_skin_condition_on_label_name'][:80]:80} -> {row['classified_condition']}")

Classification Results:
classified_condition
unclear        3530
dermatitis     1257
psoriasis       229
melanoma          7
vitiligo          6
scleroderma       4
Name: count, dtype: int64

Sample classifications:
['Inflicted skin lesions', 'Eczema', 'Irritant Contact Dermatitis']              -> dermatitis
['Prurigo nodularis', 'Actinic Keratosis', 'SCC/SCCIS']                          -> unclear
['Impetigo', 'Herpes Zoster', 'Bullous dermatitis']                              -> dermatitis
[]                                                                               -> unclear
['Lichen planus/lichenoid eruption', 'Folliculitis', 'Dermatosis due to flea']   -> unclear
['Drug Rash', 'Viral Exanthem', 'Eczema']                                        -> unclear
['Urticaria', 'Leukocytoclastic Vasculitis', 'Urticaria', 'Insect Bite', 'Leukoc -> unclear
['Inflicted skin lesions', 'Basal Cell Carcinoma', 'SCC/SCCIS']                  -> unclear
['Psoriasis', 'Eczema', 'Eczema', 'Acute a